## Saving Objects
I need to be able to save the `factored` and `homology` objects. `pickle` can save `homology`, but it doesn't work for the object type `factored` is.

#### Preliminaries

In [13]:
# load some packages
import plotly.graph_objects as go # oat uses plotly not matplotlib, theres a bit of a learning curve here but it should work fine
import pandas as pd
import oatpy as oat
import numpy as np
import pickle

# configuration
DATA_PATH = 'datasets/CCMathTopologyScavengerHunt/'

#### Setup
Make the point cloud and calculate homology.

In [14]:
# Pull the cloud
FILE = 'points2.csv'

dta = np.array(pd.read_csv(DATA_PATH + FILE, header=None)) # oat uses np arrays as coordinate inputs

# plotly plotting
trace = go.Scatter(x=dta[:, 0], y=dta[:, 1], mode='markers')
fig = go.Figure(trace)
fig.update_layout(
        width=500, 
        height=500,
        margin=dict(l=20, r=20, t=20, b=20)
    )
fig.show()

#### Homology
Find homology

In [15]:
# setup problem
enclosing = oat.dissimilarity.enclosing_from_cloud(dta) # max fintration radius
dissimilairty_matrix = oat.dissimilarity.matrix_from_cloud( # distance matrix
        cloud=dta,
        dissimilarity_max=enclosing + 1e-10 # i belive any elements past this are removed (returns a sparse matrix)
    )
# add 1e-10 to elimite some numerical error (greg says to do it)
factored = oat.rust.FactoredBoundaryMatrixVr( # two functions that do this, idk what the other one is
        dissimilarity_matrix=dissimilairty_matrix,
        homology_dimension_max=1
    )

# solve homology
homology = factored.homology( # solve homology
        return_cycle_representatives=True, # These need to be true to be able to make a barcode
        return_bounding_chains=True
    )

#### Save Everything
Idk how to do this.

In [16]:
## this works for the homology dataframe, but not for the `factored` object
filehandler = open("Gavin/testing/test.obj", "wb")
pickle.dump(homology, filehandler)
filehandler.close()

file = open("Gavin/testing/test.obj",'rb')
homology_file = pickle.load(file)
file.close()

In [21]:
## save factored
# TODO
# No idea how to do this here

factored_file = factored # this needs to not do that
# help please
# I need it

NameError: name 'factored' is not defined

In [18]:
# no cheating
del homology
del factored

#### Check Everything
Make sure we can visualize homology and calculate an optimal cycle

In [19]:
# Persistance diagram
fig = oat.plot.pd(homology_file)
fig.update_layout(
        width=600, 
        height=500,
        margin=dict(l=20, r=20, t=20, b=20)
    )
fig.show()

In [20]:
# cycle to optimize
i = homology_file[homology_file['dimension'] == 1].sort_values('cycle nnz').index[-1] # pick the largest cycle the optimize (could pick any what forms an actual cycle, this one's just easy)

# optimization problem
start = pd.Timestamp.now()
optimal = factored_file.optimize_cycle(
        birth_simplex=homology_file['birth simplex'][i], 
        problem_type='preserve PH basis'
    )
print(f'Optimizaiton took {pd.Timestamp.now() - start}')
optimal_edges = optimal['chain']['optimal cycle']['simplex'].tolist() # bounding box of optimal cycle

# plotting
trace_dta = go.Scatter( # data plot
        x=dta[:, 0],
        y=dta[:, 1],
        mode='markers',
        showlegend=True,
        name='Data',
        opacity=0.5
    )
traces_optimal = [oat.plot.edge__trace2d(edge, dta) for edge in optimal_edges] # optimal cycle plot
for n, trace in enumerate(traces_optimal): # plot optimal cycle
    trace.update(
            showlegend=(n==0),
            legendgroup='opti',
            opacity=0.5,
            name='Optimal Cycle',
            line=dict(color='black')
        )
fig = go.Figure(data=[trace_dta]+traces_optimal)
fig.update_layout(
        width=600, 
        height=500,
        margin=dict(l=20, r=20, t=20, b=20)
    )
fig.show()


Finished construcing L1 optimization program.
Constraint matrix has 9546 nonzero entries.
Passing program to solver.
Optimizaiton took 0 days 00:00:00.673720

Done solving.
MINILP solution: Solution { direction: Minimize, num_vars: 4029, num_constraints: 4384, objective: 30.28212752065371 }
